1. **Extract** all data from CSV files.
2. **Transform** data into inputs/targets.
3. **Split** inputs/targets into train/test sets.
4. **Sample** inputs/targets for the testing sets.
5. **Load** outputs inputs/targets to parquet files.

# IMPORTS

## schemas
```python

class MetadataSchema(Schema):
    """Schema for metadata in outputs."""

    timestamp: papd.Series[padt.String] = pa.Field()
    model_version: papd.Series[padt.String] = pa.Field()


class InputsSchema(Schema):
    """Schema for validating large string inputs."""

    input: papd.Series[padt.String] = pa.Field()


class OutputsSchema(Schema):
    """Schema for structured JSON outputs."""

    response: papd.Series[padt.String] = pa.Field()
    metadata: papd.Series[padt.Object] = pa.Field()


class TargetsSchema(Schema):
    """Schema for the project target."""

    input_target: papd.Series[padt.String] = pa.Field()
    response: papd.Series[padt.String] = pa.Field()

```

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

df_llm = pd.read_json("hf://datasets/Vezora/Tested-143k-Python-Alpaca/143k-Tested-Python-Alpaca-Vezora.json")
df_llm.head()

/mnt/F024B17C24B145FE/Repos/llmops-python-package/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,instruction,input,output
0,How can I add a function to find the maximum n...,,You can add the following function in your cod...
1,Devise a Python function that can detect and e...,,"Due to the complexity of the requirement, this..."
2,Please complete the modified coding challenge:...,,Here is a python solution for your coding chal...
3,You are given a code snippet that manipulates ...,,```python\nimport numpy as np\n\ndef modify_co...
4,You are tasked with implementing a job schedul...,,"```python\ndef schedule_jobs(partitions, job_s..."


In [2]:
df_input = df_llm.drop(columns=['input', 'output'])
df_input = df_input.rename(columns={'instruction': 'input'})

df_target = df_llm.drop(columns=['instruction'])
df_target = df_target.rename(columns={'output': 'response'})
df_target = df_target.rename(columns={'input': 'input_target'})

# CONFIGS

## Extract

## Transform

# Split

In [3]:
SHUFFLE = False # time-sensitive
TEST_SIZE = 0.2

# Sample

In [4]:
SAMPLE_RATIO = 0.15
SAMPLE_RANDOM_STATE = 0

## Load

In [5]:
INPUTS_TRAIN_FILE = "../data/inputs_train.parquet"
INPUTS_TEST_FILE = "../data/inputs_test.parquet"
TARGETS_TRAIN_FILE = "../data/targets_train.parquet"
TARGETS_TEST_FILE = "../data/targets_test.parquet"
INPUTS_SAMPLE_FILE = "../tests/data/inputs_sample.parquet"
TARGETS_SAMPLE_FILE = "../tests/data/targets_sample.parquet"

# SPLIT

In [6]:
inputs_train, inputs_test, targets_train, targets_test = train_test_split(
    df_input, df_target, test_size=TEST_SIZE, shuffle=SHUFFLE
)
inputs_train.shape, inputs_test.shape, targets_train.shape, targets_test.shape

((114661, 1), (28666, 1), (114661, 2), (28666, 2))

# SAMPLE

In [7]:
inputs_train_sample = inputs_train.sample(frac=SAMPLE_RATIO, random_state=SAMPLE_RANDOM_STATE)
targets_train_sample = targets_train.sample(frac=SAMPLE_RATIO, random_state=SAMPLE_RANDOM_STATE)
inputs_train_sample.shape, targets_train_sample.shape

((17199, 1), (17199, 2))

# LOAD

In [8]:
inputs_train.to_parquet(INPUTS_TRAIN_FILE)
inputs_test.to_parquet(INPUTS_TEST_FILE)
targets_train.to_parquet(TARGETS_TRAIN_FILE)
targets_test.to_parquet(TARGETS_TEST_FILE)
inputs_train_sample.to_parquet(INPUTS_SAMPLE_FILE)
targets_train_sample.to_parquet(TARGETS_SAMPLE_FILE)